In [1]:
import os

os.chdir(r'/Users/jialunlyu/Documents/GitHub/ProjectBuffalo/')

In [2]:
import buffalo.ingestion as ingestion
import buffalo.predictor as predictor

In [3]:
import torch
import pandas as pd

In [4]:
ingestor = ingestion.DataIngestion(ingestion.enum.API.ADVANTAGE)

In [26]:
import pandas as pd

td = pd.Timedelta(minutes=1, seconds=10, microseconds=1)

td.resolution_string

'U'

In [5]:
ingestor.load_data(r'cached_data/ingestion.sqlite')

In [6]:
timestamps = ingestor.data['ADJUSTED_DAILY_STOCK'].query('symbol == "AAPL"').index

In [ ]:
stock_data = ingestor.data['ADJUSTED_DAILY_STOCK']
for k, v in ingestor.data.items():
    if k != 'ADJUSTED_DAILY_STOCK':
        stock_data = predictor.util.align_dataframe_by_time(stock_data, v)